In [4]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt
import pandas as pd
import sys

In [5]:
df_iris = pd.read_csv(u'data/iris.txt',sep=' ')

In [6]:
A = np.hstack([
        np.matrix(df_iris.sl).T, 
        np.matrix(df_iris.sw).T, 
        np.matrix(df_iris.pl).T, 
        np.matrix(df_iris.pw).T])

In [7]:
c = np.matrix(df_iris.c).T

In [8]:
## Use a heap to store the smallest items

import heapq as hp

# Define an object and overload custom comparison operators / tuple object created by @atcemgil
class tup:
    def __init__(self, val, idx):
        self.val = val
        self.idx = idx
        
    def __lt__(self, other):
        '''Redefine for max-heap'''
        return self.val > other.val
    
    def __le__(self, other):
        return self.val <= other.val
 
    def __eq__(self, other):
        return self.val == other.val
    
    def __ne__(self, other):
        return self.val != other.val

    def __gt__(self, other):
        return self.val > other.val

    def __ge__(self, other):
        return self.val >= other.val

    def __str__(self):
        return '{:.3},{:d}'.format(self.val,self.idx)

In [17]:
def K_NN(k, A, x):
    K = k
    heap = []
    N = A.shape[0]
    
    dummy_tupple = tup(np.inf, -1) #with value infinity and at index -1
    
    for i in range(K):
        hp.heappush(heap, dummy_tupple)
        
    for j in range(N):
        e = A[j,:]- x
        e = e.reshape((4,1))
        tp = tup(float(e.T*e), j)
        if tp <= heap[0]:
            hp.heapreplace(heap, tp)
    
    for m in range(K):
        h = hp.heappop(heap)
        print('{:.3}'.format(h.val), float(c[h.idx]))

#### The values collected in the heap for any given value can be printed using the K_NN function:

In [18]:
x_test = A[1,:] + 5*np.random.randn(1,4)
K_NN(3, A, x_test)

20.9 3.0
20.2 3.0
17.3 3.0


In [117]:
def EvauluteWithLeaveOneOut(k, A, t):
    K = k
    heap = []
    N = A.shape[0]
    
    W = np.matrix(np.diag([0,0,0,0]))
    
    false_positive = 0
    false_negative = 0
    true_positive = 0
    true_negative = 0

    dummy_tupple = tup(np.inf, -1) #with value infinity and at index -1
    
    for i in range(K):
        hp.heappush(heap, dummy_tupple)
    
    
    for i in range(N):
        
        ## for every other element than i calculate the heap
        for j in range(N):
            if i != j:
                e = A[i,:] - A[j,:]
                e = e.reshape((4,1))
                tp = tup(float(e.T*W*e), i)
                if tp <= heap[0]:
                    hp.heapreplace(heap, tp)
        
        sumAll = 0
        for h in range(K):
            sumAll += c[hp.heappop(heap).idx]
        
        ## reset the heap beacuse values are pushed
        for m in range(K):
            hp.heappush(heap, dummy_tupple)
        
        average = round(sumAll.item(0)/K)

        if (c[i] == t):
            if(average == t):
                true_positive += 1
            else:
                false_negative += 1
        else:
            if(average == t):
                false_positive += 1
            else:
                true_negative += 1
    
    print(" ")
    print("type",t, "versus the others:")
    print("accuracy: ")
    print(1.*(true_positive+true_negative)/N)
    print("precision: ")
    print(1.*true_positive/(true_positive+false_positive))
    print("recall: ")
    print(1.*true_positive/(true_positive+false_negative))

#### calculate accuracy, precision and recall for 1, 2 or 3 versus the others:

In [118]:
EvauluteWithLeaveOneOut(3, A, 1)
EvauluteWithLeaveOneOut(3, A, 2)
EvauluteWithLeaveOneOut(3, A, 3)

 
type 1 versus the others:
accuracy: 
1.0
precision: 
1.0
recall: 
1.0
 
type 2 versus the others:
accuracy: 
1.0
precision: 
1.0
recall: 
1.0
 
type 3 versus the others:
accuracy: 
1.0
precision: 
1.0
recall: 
1.0
